[View in Colaboratory](https://colab.research.google.com/github/NaNkotsukan/deepLearningWithChainer/blob/master/deeplearningWithChainer.ipynb)

## 環境設定

In [0]:
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install cupy-cuda80 chainer matplotlib
!nvidia-smi

# MNISTデータセットのクラス分類
MNISTデータセットを用いてクラス分類をします．深い理解がなくても進められると思うので頑張っていきましょう．

## 深く知りたい?

「ゼロから作るDeep Learning」って本おすすめ．

#### 更に深くって?
「パターン認識と機械学習」って本が凄いらしい．

## Chainerのインポート

使うライブラリを読み込む．

Chainerは国産の機械学習ライブラリで全てPythonで書かれてるので中を見ることができる．

公式ドキュメント：https://docs.chainer.org/en/stable/index.html


In [0]:
from chainer import Chain, serializers, computational_graph
from chainer import Variable as V
import chainer.functions as F
import chainer.links as L
import chainer.computational_graph as C
from chainer.optimizers import Adam, SGD
import chainer

# import pydot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image, display

import os
import numpy as np
import cupy as cp

%matplotlib inline

### データセットを読み込む

MNISTデータセットをメモリ上に読み込む.

50000枚の学習用データセットと10000枚のテスト用データセットを読み込む

In [0]:
train, test = chainer.datasets.get_mnist(ndim=2)
trainX = train._datasets[0]
trainT = train._datasets[1]
testX = test._datasets[0]
testT = test._datasets[1]

print(trainX.shape)
print(trainT.shape)
print(testX.shape)
print(testT.shape)

In [0]:
def show_img(img, title):
    fig = plt.figure(figsize=(15, 20))
    for i in range(len(img)):
        a = fig.add_subplot(1, len(img), i+1)
        a.imshow(img[i])
        plt.xlim((0, 28))
        plt.ylim((28, 0))
        plt.tick_params(labelbottom=False, labelleft=False)
        plt.title(title[i], fontsize=15)
        plt.grid()
    plt.show()

In [0]:
show_img(trainX[0:10], trainT[0:10])

### ニューラルネットワーク

まずはじめに二層の全結合層(Affine変換)のみで構成したニューラルネットワークを学習させてみる．

In [0]:
class Model0(Chain):
    def __init__(self):
        super(Model0, self).__init__()
        with self.init_scope():
            self.l0 = L.Linear(784, 500)
#           self.add_link("l0", L.Linear(784, 500))     # このようにメタな書き方をすることも出来る．上と等価
            self.l1 = L.Linear(500, 300)

    def __call__(self, x):
        h = self.l0(x)
#       h = self["l0"](x)       # このようにメタな書き方をすることも出来る．上と等価
        h = F.relu(h)
        h = self.l1(h)
        return h

## 定義したモデルを学習させる
### 学習の準備

Optimizerはモデルの学習におけるパラメータを更新してくれるクラス．

[optimizers](http://docs.chainer.org/en/stable/reference/optimizers.html#optimizers)にはSGD, RMSprop, AdaGrad，Adam，などいろいろあるけど今回はよく用いられるAdamを使う．


In [0]:
def test(model, x, t):
    y = model(x)
    loss = F.softmax_cross_entropy(y, t)
    acc = F.accuracy(y, t)
    return loss.data, acc.data

def batch(model, optimizer, x, t):
    y = model(x)
    loss = F.softmax_cross_entropy(y, t)
    model.cleargrads()
    loss.backward()
    optimizer.update()
    return loss.data

def training(model, epoch, batchsize, trainX, trainT, testX, testT):
    optimizer = Adam()
    optimizer.setup(model)
    for i in range(epoch):
        index = np.random.permutation(len(trainX))
        for j in range(0, len(trainX), batchsize):
            x = np.asarray(trainX[index[j:j+batchsize]]).astype(np.float32)
            t = np.asarray(trainT[index[j:j+batchsize]])
            loss = batch(model, optimizer, x, t)
            if j%10000==0:
                print(f"epoch:{i:3d} batch:{j:6d} loss:{float(loss):0.4f}")
        x = np.asarray(testX).astype(np.float32)
        t = np.asarray(testT)
        loss, acc = test(model, x, t)
        print(f"epoch:{i:3d} testLoss:{float(loss):0.4f} testAccuracy:{float(acc):0.4f}")
    model.to_cpu()


def training_GPU(model, epoch, batchsize, trainX, trainT, testX, testT):
    model.to_gpu()
    optimizer = Adam()
    optimizer.setup(model)
    for i in range(epoch):
        index = np.random.permutation(len(trainX))
        for j in range(0, len(trainX), batchsize):
            x = cp.asarray(trainX[index[j:j+batchsize]]).astype(np.float32)
            t = cp.asarray(trainT[index[j:j+batchsize]])
            loss = batch(model, optimizer, x, t)
            if j%10000==0:
                print(f"epoch:{i:3d} batch:{j:6d} loss:{float(loss):0.4f}")
        x = cp.asarray(testX).astype(np.float32)
        t = cp.asarray(testT)
        loss, acc = test(model, x, t)
        print(f"epoch:{i:3d} testLoss:{float(loss):0.4f} testAccuracy:{float(acc):0.4f}")
    model.to_cpu()

### 学習を実行する
epoch数，batchsizeを設定する．1epochとはデータセットを全て1通り学習することである．1epoch終えるとデータセットをシャッフルし次のepochに入る．batchsizeは1度に学習するデータの数のことである．

In [0]:
epoch = 2
batchsize = 100
model = Model0()
training(model, epoch, batchsize, trainX, trainT, testX, testT)

In [0]:
result = model(testX[:100]).data.argmax(axis=1)
for i in range(0, 30, 10):
    show_img(testX[i:i+10], [f"t:{testT[i+j]}  y:{result[i+j]}" for j in range(10)])

### 精度の改善

epoch数，batchsize，modelの層の数を変えてみよう．

In [0]:
class Model1(Chain):
    def __init__(self):
        super(Model1, self).__init__()
        with self.init_scope():
            self.l0 = L.Linear(784, 500)
            self.l1 = L.Linear(500, 300)
            self.l2 = L.Linear(300, 100)
            self.l3 = L.Linear(100, 10)            

    def __call__(self, x):
        h = F.relu(self.l0(x))
        h = F.relu(self.l1(h))
        h = F.relu(self.l2(h))
        h = self.l3(h)
        return h

In [0]:
epoch = 10
batchsize = 100
model = Model1()
training_GPU(model, epoch, batchsize, trainX, trainT, testX, testT)

### 更に精度を上げる

畳み込みニューラルネットワーク(CNN)を使用する．
model0，model1では線形結合層のみでモデルを定義した．しかし深層学習の火付け役となったAlexNet登場以降，CNNが常用される．画像認識などの分野でとても力を発揮するので使ってみよう．

In [0]:
class Model2(Chain):
    def __init__(self):
        super(Model2, self).__init__()
        with self.init_scope():
            self.conv0 = L.Convolution2D(1, 32, ksize=3)
            self.conv1 = L.Convolution2D(32, 32, ksize=3)
            self.conv2 = L.Convolution2D(32, 32, ksize=3)
            self.conv3 = L.Convolution2D(32, 32, ksize=3)
            self.l0 = L.Linear(2048 , 10)

    def __call__(self, x):
        h = F.reshape(x, (-1, 1, 28, 28))
        h = F.relu(self.conv0(h))
        h = F.relu(self.conv1(h))
        h = F.max_pooling_2d(h, 2)
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        h = self.l0(h)
        return h

In [0]:
epoch = 10
batchsize = 100
model = Model2()
training_GPU(model, epoch, batchsize, trainX, trainT, testX, testT)

###もっと精度上げたい?
#### さらに深く
https://arxiv.org/abs/1512.03385
#### もっと深く
https://arxiv.org/abs/1806.05393